Making Neural Network from Scratch using plain python with **Insurance Dataset**

In [18]:
import numpy as np
import pandas as pd

# Loading dataset

In [19]:
df = pd.read_csv(r'/content/drive/MyDrive/Documents/Datasets from youtube/insurance_data (1).csv')
print(df.shape)
df.head()

(28, 3)


,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


# Checking any Null value

In [20]:
df.isnull().sum()

age                 0
affordibility       0
bought_insurance    0
dtype: int64

# With No Null value we can move onto spliting our dataset into traning and testing parts using **train_test_split**

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age' , 'affordibility']] , df['bought_insurance'] , test_size=0.2 , random_state=25)
X_test.head()

,age,affordibility
2,47,1
10,18,1
21,26,0
11,28,1
14,49,1


# For better efficency we do scaling

In [22]:
# suppose our age is from 1 to 100
X_train_scalled = X_train.copy()
X_test_scalled = X_test.copy()

X_train_scalled['age'] = X_train['age']/100
X_test_scalled['age'] =X_test['age']/100
X_test_scalled.head()

,age,affordibility
2,0.47,1
10,0.18,1
21,0.26,0
11,0.28,1
14,0.49,1


In [23]:
X_train_scalled.head()

,age,affordibility
0,0.22,1
13,0.29,0
6,0.55,0
17,0.58,1
24,0.50,1


# Making Neural Network from Scratch

In [24]:
import numpy as np
class Neural_Network:
    def __init__(self): # self is a keyword and is used for calling function in a class or variables define in a constructor
        self.w1 = 1
        self.w2 = 1
        self.bias = 0

    def sigmoid(self , ws): # giving it a sigmoid activation function
        result = 1 / (1+np.exp(-ws))
        return result

    def log_loss(self , y_predicted , y_true): # caculating loss
        epsilon = 1e-15
        y_predicted_new = [max(i,epsilon) for i in y_predicted]
        y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
        y_predicted_new = np.array(y_predicted_new)
        return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))


    def gradient_descent(self , age , affordability , y_truth , epochs):

        for i in range(epochs):
            weighted_sum = self.w1 * age + self.w2 * affordability + self.bias
            y_predicted = self.sigmoid(weighted_sum)

            loss = self.log_loss(y_predicted , y_truth)

            # now we take the derivative to update the values of weights and bias
            learning_rate = 0.5

            n = len(age)

            w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_truth))
            w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_truth))

            bias_d = np.mean(y_predicted-y_truth)


            learning_rate = 0.1
            self.w1 = self.w1 - learning_rate * w1d
            self.w2 = self.w2 - learning_rate * w2d
            self.bias = self.bias - learning_rate * bias_d

            # print weights after every 50 epochs
            if i%50 == 0:
                print(f'epoch : {i} weight1 : {self.w1} , weight2 : {self.w2} , bias : {self.bias} , loss : {loss}')


            # print weights on the last iteration
            if i==epochs-1:
                print(f'weight1 : {self.w1} , weight2 : {self.w2} , bias : {self.bias} , loss : {loss}')

        # as we want to get the weights and bias when we do model.fit so
        return self.w1,self.w2,self.bias

    def fit(self, X, Y, epochs):
        return self.gradient_descent(X['age'] , X['affordibility'] , Y , epochs)

    def predict(self, X_test):
        weighted_sum = self.w1*X_test['age'] + self.w2*X_test['affordibility'] + self.bias
        return self.sigmoid(weighted_sum)

# Traning our model

In [25]:
model = Neural_Network()
model.fit(X_train_scalled, y_train , 8000)

epoch : 0 weight1 : 0.9949815266940354 , weight2 : 0.9896696250789058 , bias : -0.022683735472737165 , loss : 0.7113403233723417
epoch : 50 weight1 : 0.9500930885415674 , weight2 : 0.804909020986925 , bias : -0.6409684073086737 , loss : 0.605840882528726
epoch : 100 weight1 : 1.0651807066987198 , weight2 : 0.8712141663478571 , bias : -0.8544393593687205 , loss : 0.5925483296513601
epoch : 150 weight1 : 1.2051066705614086 , weight2 : 0.9595761602192402 , bias : -0.994388310973579 , loss : 0.5831116696004638
epoch : 200 weight1 : 1.348404774050644 , weight2 : 1.0368550412385873 , bias : -1.1137591957694988 , loss : 0.5749386575004722
epoch : 250 weight1 : 1.4916724882233876 , weight2 : 1.1011257122864226 , bias : -1.2215953578871015 , loss : 0.5676659367618607
epoch : 300 weight1 : 1.6340903405280722 , weight2 : 1.1542608622990649 , bias : -1.3205556736951605 , loss : 0.5610743352916638
epoch : 350 weight1 : 1.7752701340123371 , weight2 : 1.1982396875981078 , bias : -1.4121465205758779 ,

(9.553588216731189, 1.7428159278312176, -5.046260460148606)

In [26]:
y_pred = model.predict(X_test_scalled)
y_pred

2     0.766158
10    0.170258
21    0.071603
11    0.347864
14    0.798640
9     0.925824
dtype: float64

In [17]:
[np.argmax(i) for i in y_pred]

[0, 0, 0, 0, 0, 0]

In [27]:
y_test

2     1
10    0
21    0
11    0
14    1
9     1
Name: bought_insurance, dtype: int64

In [31]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test , [np.argmax(i) for i in y_pred]) # accuracy of our model

0.5

# As for lower dataset we still got our accuracy 50%